# Test GradientBoostingClassifier e RandomForestClassifier sui dati processati con il metodo proposto

In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

### Utility functions

In [2]:
def print_classification_report(clf, X_test, y,test):
    y_pred = clf.predict(X_test)
    print(classification_report(y_test,y_pred))

# Dataset completo 
Concatenazione delle feature

In [3]:
df = pd.read_csv("csv/dataset_eeg_nseg.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [4]:
df

0         1          2          3          4  \
id_user session video                                                        
10      1       1      0.682185  6.574546  30.976157  43.943447  33.897774   
                2      2.286622  1.901831  10.923501  14.502541  13.116158   
                3      2.328174  8.771254  21.033531  27.952410  15.763804   
                4      1.445019  2.531786   8.283509  16.000674  21.283076   
                5      1.785984  3.370043  13.056309  21.372322  14.467699   
...                         ...       ...        ...        ...        ...   
9       3       20     0.018484  0.073685   0.338801   0.326051   0.133009   
                21     0.026977  0.069479   0.336707   0.554167   0.940006   
                22     0.016623  0.079514   0.268469   0.597706   0.518611   
                23     0.096016  0.358913   0.799127   0.678415   0.692425   
                24     0.030325  1.154748   1.514104   1.716616   1.489050   

                               5          6          7          8         9  \
id_user session video                                                         
10      1       1      31.408804  13.454868   5.417151   8.311784  3.687850   
                2       8.780469  10.822784  10.149996   8.758160  6.773942   
                3       7.873453   4.418694   8.135071  11.337191  6.107161   
                4      10.855781   5.381098   8.396707   6.322739  7.215805   
                5       6.821163   4.805254   3.786888   1.078734  5.213206   
...                          ...        ...        ...        ...       ...   
9       3       20      0.595424   0.526718   0.209873   0.486814  0.422881   
                21      1.297052   0.865415   0.288125   0.251682  0.294775   
                22      0.180433   0.298493   0.193067   0.403564  0.386624   
                23      0.459945   0.624406   0.304234   0.478095  0.476208   
                24      0.630518   0.385306   0.914131   1.532081  1.864732   

                       ...      5502      5503      5504      5505      5506  \
id_user session video  ...                                                     
10      1       1      ...  0.393595  0.288753  0.301594  0.330172  0.382520   
                2      ...  0.220979  0.304374  0.258954  0.226489  0.278128   
                3      ...  0.398413  0.428280  0.397462  0.411529  0.419848   
                4      ...  0.401126  0.542530  0.498371  0.345471  0.304640   
                5      ...  0.608174  0.375220  0.135943  0.186317  0.573142   
...                    ...       ...       ...       ...       ...       ...   
9       3       20     ...  0.543079  0.295776  0.314458  0.431749  0.504237   
                21     ...  0.584304  0.382018  0.415148  0.429720  0.411082   
                22     ...  0.336165  0.521748  0.560112  0.353164  0.442433   
                23     ...  0.472622  0.503611  0.484663  0.454322  0.631501   
                24     ...  0.673989  0.666538  0.671457  0.663296  0.452001   

                           5507      5508      5509      5510  emotion  
id_user session video                                                   
10      1       1      0.395456  0.253277  0.323946  0.301332        1  
                2      0.338855  0.263531  0.405918  0.678865        2  
                3      0.515295  0.400210  0.367307  0.432373        3  
                4      0.548280  0.597810  0.545700  0.518481        0  
                5      0.422333  0.380185  0.548570  0.433106        2  
...                         ...       ...       ...       ...      ...  
9       3       20     0.595706  0.355203  0.431082  0.533335        0  
                21     0.448554  0.678730  0.567217  0.346940        2  
                22     0.506206  0.434594  0.224685  0.230687        0  
                23     0.659767  0.452515  0.524196  0.552201        1  
                24     0.124448  0.092329  0.239485  0.409435        0

## Split train e test
### Subject Dependent
16 trials train e 8 test
2 testing group for labels 

In [13]:
def subject_dependent_split(df,session):
    df = df.reset_index()
    np.random.seed(75)
    test_videos = np.random.choice(np.arange(1, 25), replace=False, size=(8))
    df_sess = df.loc[df['session'] == session]
    X_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    X_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    return X_train, X_test, y_train, y_test

In [14]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=30, learning_rate=0.1, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [15]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.30833333333333335
TRAIN: 1.0


In [16]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=7,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [17]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.28611111111111115
TRAIN: 0.9569444444444445


### Subject Indipendent
Leave One Out Cross Validation sugli utenti

In [18]:
def subject_independent_split(df,session):
    df = df.reset_index()
    df_sess = df.loc[df['session'] == session]
    groups = df_sess['id_user']
    X = df_sess.set_index(['id_user','session','video']).drop('emotion',axis=1)
    y = df_sess.set_index(['id_user','session','video']).emotion
    return X, y, groups

In [19]:
logo = LeaveOneGroupOut()

In [20]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_gbc.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 10.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 10.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 10.1min finished


In [21]:
print("TEST:", np.array(scores_test).mean())

TEST: 0.3518518518518519


In [23]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_rfc.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.9s finished


In [24]:
print("TEST:", np.array(scores_test).mean())

TEST: 0.2916666666666667


### Subject-Biased Experiment
train_test_split

In [25]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [38]:
clf = GradientBoostingClassifier(n_estimators=25, learning_rate=0.001, max_depth=30, random_state=42).fit(X_trainSB, y_trainSB)

In [39]:
print("TEST", clf.score(X_testSB, y_testSB))
print("train", clf.score(X_trainSB, y_trainSB))

TEST 0.24074074074074073
train 1.0


In [40]:
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22).fit(X_trainSB, y_trainSB)

In [41]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("train", rfc.score(X_trainSB, y_trainSB))

TEST 0.3287037037037037
train 0.9780092592592593


# Esecuzione su singole feature 
Dataset con singola feature tra PSD, DE, ASM, DAMS
## PSD

In [42]:
df = pd.read_csv("csv/dataset_eeg_psd_stand.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [43]:
df

0         1          2          3          4  \
id_user session video                                                        
10      1       1      0.387444  4.431951  21.181176  30.081901  23.186569   
                2      3.613461  2.987054  17.673562  23.499934  21.243021   
                3      2.538314  9.836678  23.726705  31.564027  17.757451   
                4      1.857322  3.333541  11.146437  21.629107  28.804499   
                5      2.890081  5.532890  21.693239  35.567487  24.047969   
...                         ...       ...        ...        ...        ...   
9       3       20     0.379379  2.002579   9.798390   9.423484   3.747013   
                21     0.261015  0.941350   5.218861   8.699744  14.875869   
                22     0.248579  1.993261   7.235135  16.368623  14.174425   
                23     1.832752  7.285489  16.415975  13.912286  14.202871   
                24     0.079021  8.721247  11.483227  13.039720  11.290664   

                               5          6          7          8          9  \
id_user session video                                                          
10      1       1      21.478144   9.154592   3.637517   5.624388   2.450527   
                2      14.184890  17.509604  16.414363  14.148572  10.918434   
                3       8.819698   4.906340   9.116045  12.743229   6.818941   
                4      14.640503   7.203926  11.300201   8.483011   9.696114   
                5      11.290661   7.927364   6.228345   1.710122   8.607982   
...                          ...        ...        ...        ...        ...   
9       3       20     17.344477  15.324158   6.007236  14.150769  12.270789   
                21     20.591107  13.681895   4.441211   3.857870   4.547664   
                22      4.792880   8.068051   5.143370  10.982866  10.512923   
                23      9.380996  12.792094   6.151397   9.757453   9.718315   
                24      4.692054   2.807376   6.871881  11.621400  14.178128   

                       ...      2997      2998      2999      3000      3001  \
id_user session video  ...                                                     
10      1       1      ... -0.080806 -0.080806 -0.080806 -0.080806 -0.080806   
                2      ... -0.108965 -0.108965 -0.108965 -0.108965 -0.108965   
                3      ... -0.098912 -0.098912 -0.098912 -0.098912 -0.098912   
                4      ... -0.105530 -0.105530 -0.105530 -0.105530 -0.105530   
                5      ... -0.089614 -0.089614 -0.089614 -0.089614 -0.089614   
...                    ...       ...       ...       ...       ...       ...   
9       3       20     ... -0.164155 -0.164155 -0.164155 -0.164155 -0.164155   
                21     ... -0.170800 -0.170800 -0.170800 -0.170800 -0.170800   
                22     ... -0.212575 -0.212575 -0.212575 -0.212575 -0.212575   
                23     ... -0.158724 -0.158724 -0.158724 -0.158724 -0.158724   
                24     ... -0.154057 -0.154057 -0.154057 -0.154057 -0.154057   

                           3002      3003      3004      3005  emotion  
id_user session video                                                   
10      1       1     -0.080806 -0.080806 -0.080806 -0.080806        1  
                2     -0.108965 -0.108965 -0.108965 -0.108965        2  
                3     -0.098912 -0.098912 -0.098912 -0.098912        3  
                4     -0.105530 -0.105530 -0.105530 -0.105530        0  
                5     -0.089614 -0.089614 -0.089614 -0.089614        2  
...                         ...       ...       ...       ...      ...  
9       3       20    -0.164155 -0.164155 -0.164155 -0.164155        0  
                21    -0.170800 -0.170800 -0.170800 -0.170800        2  
                22    -0.212575 -0.212575 -0.212575 -0.212575        0  
                23    -0.158724 -0.158724 -0.158724 -0.158724        1  
                24    -0.154057 -0.154057 -0.154057 -0.15

### Subject Dependent

In [44]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=80, learning_rate=0.001, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [45]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.18611111111111112
TRAIN: 0.6625


In [46]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [47]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.29444444444444445
TRAIN: 0.9694444444444444


### Subject indipendent

In [50]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_gbc.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  6.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  6.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  6.4min finished


In [51]:
print("TEST:", np.array(scores_test).mean())

TEST: 0.3453703703703704


In [53]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_rfc.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.8s finished


In [54]:
print("TEST:", np.array(scores_test).mean())

TEST: 0.30000000000000004


### Subject Biased

In [55]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [56]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [57]:
print("TEST", clf.score(X_testSB, y_testSB))
print("TEST", clf.score(X_trainSB, y_trainSB))

TEST 0.2962962962962963
TEST 0.6643518518518519


In [58]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [59]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("TEST", rfc.score(X_trainSB, y_trainSB))

TEST 0.3333333333333333
TEST 0.65625


## DE

In [60]:
df = pd.read_csv("csv/dataset_eeg_de_stand.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [61]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      2.489523  4.344901  5.170953  5.342168  5.188602   
                2      3.262889  3.717398  4.562123  4.868865  4.522181   
                3      3.246484  4.594690  5.209708  5.140998  4.805019   
                4      3.287889  3.562657  4.222406  4.615180  4.805029   
                5      2.762618  3.649876  4.481365  4.691567  4.536483   
...                         ...       ...       ...       ...       ...   
9       3       20     1.335110  2.546374  3.679002  3.642087  2.985341   
                21     1.578924  1.982117  2.792630  3.346528  3.617193   
                22     1.227010  2.259839  2.991675  3.501839  3.451462   
                23     2.566158  3.349784  3.886221  3.826749  3.852578   
                24     1.506018  3.029799  3.334757  3.921628  3.732247   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1      4.956505  4.411793  3.974491  4.070616  3.840260  ...   
                2      4.326004  3.820923  3.794147  3.828815  4.057891  ...   
                3      4.634409  4.120910  4.122282  4.230265  4.064393  ...   
                4      4.448752  3.914953  3.902403  3.556226  3.650531  ...   
                5      4.516854  4.044487  4.203614  3.494943  3.826665  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     3.905410  3.907160  2.871236  3.539994  3.759385  ...   
                21     3.734571  3.409892  2.816747  2.784841  2.799835  ...   
                22     2.809616  3.045351  2.675630  3.237452  3.168802  ...   
                23     3.633987  3.705771  3.212770  3.566951  3.591921  ...   
                24     3.268741  3.724833  4.076077  4.236571  4.298587  ...   

                            492       493       494       495       496  \
id_user session video                                                     
10      1       1     -0.839828  0.092095  0.412395  0.067338  0.001055   
                2     -0.341535  0.219842  0.455068  0.756253 -0.008140   
                3      0.367681  0.401783  0.358794  0.359843  0.286991   
                4     -0.693555 -0.942542 -0.824353 -0.548958 -0.478757   
                5     -0.479519 -0.154385 -0.058944 -0.587805 -0.984136   
...                         ...       ...       ...       ...       ...   
9       3       20    -0.448799 -0.405123 -0.522088 -0.586491 -0.919928   
                21    -0.741436 -0.850986 -0.925450 -0.784589 -0.598273   
                22    -0.572512 -0.654173 -0.858788 -0.863210 -0.671390   
                23    -0.496109 -0.366960 -0.598830 -1.041940 -1.165133   
                24    -0.712604 -0.141463 -0.043210 -0.297809 -0.618274   

                            497       498       499       500  emotion  
id_user session video                                                   
10      1       1     -0.071205 -0.195604 -0.084020 -0.897776        1  
                2     -0.065216  0.411846 -0.282226 -1.545442        2  
                3      0.118580  0.458729  0.698105  0.316571        3  
                4     -0.794974 -0.908540 -1.006668 -1.469874        0  
                5     -0.379504 -0.773932 -0.677215 -0.742828        2  
...                         ...       ...       ...       ...      ...  
9       3       20    -0.695577 -0.432950 -0.520635 -0.942513        0  
                21    -0.601112 -0.731575 -0.896092 -1.492268        2  
                22    -0.914961 -0.774533 -1.091544 -1.308467        0  
                23    -0.958110 -0.729841 -0.623746 -1.210566        1  
                24    -0.688694 -0.706654 -0.853856 -1.465197        0  

[1080 rows x 502 columns]

### Subject Dependent

In [62]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=80, learning_rate=0.001, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [63]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.24444444444444446
TRAIN: 0.7458333333333332


In [64]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [65]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.325
TRAIN: 0.9819444444444444


### Subject Independent

In [67]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_gbc.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.2min finished


In [68]:
print("TEST:", np.array(scores_test).mean())

TEST: 0.31666666666666665


In [69]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_rfc.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.3s finished


In [70]:
print("TEST:", np.array(scores_test).mean())

TEST: 0.3092592592592593


### Subject-Biased

In [71]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [72]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [73]:
print("TEST", clf.score(X_testSB, y_testSB))
print("TEST", clf.score(X_trainSB, y_trainSB))

TEST 0.25
TEST 0.6655092592592593


In [74]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [75]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("TEST", rfc.score(X_trainSB, y_trainSB))

TEST 0.3194444444444444
TEST 0.6782407407407407


## ASM

In [76]:
df = pd.read_csv("csv/dataset_eeg_asm_stand.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [77]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      1.012088 -0.911560 -0.665793 -0.500183 -0.042620   
                2      0.537768 -1.442027 -0.074195  0.092136  0.879269   
                3     -0.385238 -1.137230 -0.764816  0.339483  0.307085   
                4     -2.105762 -2.000142 -0.885856 -0.926382 -1.119494   
                5      0.333797  0.116966  0.534304  0.752024  0.614429   
...                         ...       ...       ...       ...       ...   
9       3       20     0.828222  0.110005 -0.292211 -0.256771 -0.093193   
                21    -0.816994 -0.426948  1.019062 -0.376050 -0.074896   
                22     0.296071 -0.220305  0.380152  0.772413  0.556812   
                23     0.274636  0.536893  0.470476  0.174330  0.160432   
                24    -1.067859  1.961140  0.507691 -0.009959  0.731412   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1      0.663276  0.823349  0.514256  1.227172  0.350683  ...   
                2      0.345412  1.207485  1.504276  1.659325  1.059000  ...   
                3     -1.452303 -0.926742  0.743929  1.185932  0.562245  ...   
                4     -1.160876 -0.504920  0.097972  0.735077  0.773215  ...   
                5      0.108085  0.997369 -0.862991 -0.463013  0.657782  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     0.390836  0.073658  1.217579  1.064883  0.033453  ...   
                21     0.447525  0.896940  0.246104 -0.031682  0.679981  ...   
                22     0.266610  0.697412  0.923660  0.791160  0.993296  ...   
                23    -0.066678  0.546428  0.746530  0.472161  0.302017  ...   
                24    -0.596442 -2.021336 -1.192335 -0.378628  0.003359  ...   

                            492       493       494       495       496  \
id_user session video                                                     
10      1       1     -0.639778 -1.202271 -1.133376 -0.980049 -0.699199   
                2     -1.351574 -0.911699 -1.151272 -1.322512 -1.050138   
                3     -0.670586 -0.463313 -0.677185 -0.579566 -0.521829   
                4     -0.867621 -0.024456 -0.287766 -1.199479 -1.442944   
                5      0.573537 -0.777506 -2.165219 -1.873071  0.370365   
...                         ...       ...       ...       ...       ...   
9       3       20     0.519902 -0.861157 -0.756824 -0.101817  0.302991   
                21     0.884594 -0.788778 -0.514719 -0.394174 -0.548347   
                22    -0.845886  0.557844  0.848024 -0.717306 -0.042088   
                23     0.062857  0.296372  0.153592 -0.075043  1.260092   
                24     1.300840  1.251262  1.283991  1.229693 -0.176186   

                            497       498       499       500  emotion  
id_user session video                                                   
10      1       1     -0.629793 -1.392603 -1.013454 -1.134784        1  
                2     -0.729830 -1.127129 -0.376100  1.063578        2  
                3      0.140544 -0.658113 -0.886450 -0.434915        3  
                4      0.009831  0.305169 -0.005550 -0.167853        0  
                5     -0.504267 -0.748713  0.227857 -0.441791        2  
...                         ...       ...       ...       ...      ...  
9       3       20     0.813798 -0.529287 -0.105538  0.465489        0  
                21    -0.238373  1.665719  0.743251 -1.078958        2  
                22     0.440286 -0.101379 -1.689103 -1.643707        0  
                23     1.473086 -0.088661  0.451489  0.662521        1  
                24    -2.355607 -2.569316 -1.590194 -0.459408        0  

[1080 rows x 502 columns]

### Subject Dependent

In [78]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=20, learning_rate=0.001, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [79]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.18055555555555558
TRAIN: 0.5027777777777778


In [80]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [81]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.2333333333333333
TRAIN: 0.9902777777777777


### Subject indipendent

In [83]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_gbc.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.8min finished


In [84]:
print("TEST:", np.array(scores_test).mean())

TEST: 0.2685185185185185


In [86]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_rfc.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.5s finished


In [87]:
print("TEST:", np.array(scores_test).mean())

TEST: 0.2537037037037037


### Subject Biased

In [88]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [89]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [90]:
print("TEST", clf.score(X_testSB, y_testSB))
print("TEST", clf.score(X_trainSB, y_trainSB))

TEST 0.2175925925925926
TEST 0.6342592592592593


In [91]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [92]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("TEST", rfc.score(X_trainSB, y_trainSB))

TEST 0.2222222222222222
TEST 0.7118055555555556


## DASM

In [93]:
df = pd.read_csv("csv/dataset_eeg_dasm_stand.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [94]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      0.460587 -1.661409 -1.339916 -1.193845 -0.806338   
                2     -0.367031 -2.488707 -0.917907 -0.772364 -0.056634   
                3     -1.113704 -1.744784 -1.446346 -0.593374 -0.625481   
                4     -2.797332 -2.809042 -1.529033 -1.555901 -1.735630   
                5     -0.530895 -0.686026 -0.355869 -0.182906 -0.261843   
...                         ...       ...       ...       ...       ...   
9       3       20    -0.285193 -1.191839 -1.757393 -1.701040 -1.458603   
                21    -1.933472 -1.400307 -0.261141 -1.514140 -1.217411   
                22    -0.961390 -1.376941 -0.839137 -0.526375 -0.726126   
                23    -0.931552 -0.667086 -0.735599 -1.018452 -1.039439   
                24    -2.277371  0.479542 -0.483347 -0.948508 -0.348146   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1     -0.203387 -0.024879 -0.254702  0.388953 -0.405003  ...   
                2     -0.525367  0.097267  0.326794  0.480073  0.080292  ...   
                3     -2.070992 -1.605592 -0.302456 -0.023266 -0.434847  ...   
                4     -1.833796 -1.174420 -0.611973 -0.132899 -0.103544  ...   
                5     -0.596853  0.237978 -1.747574 -1.381626 -0.258679  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20    -0.821914 -1.255126  0.100247 -0.027388 -1.298510  ...   
                21    -0.736597 -0.354187 -0.882931 -1.142031 -0.531049  ...   
                22    -1.004105 -0.588651 -0.385381 -0.510356 -0.336515  ...   
                23    -1.274275 -0.659671 -0.482453 -0.741357 -0.904608  ...   
                24    -1.480368 -1.213595 -2.177121 -1.616666 -1.156540  ...   

                           1494      1495      1496      1497      1498  \
id_user session video                                                     
10      1       1      1.152843  1.152843  1.152843  1.152843  1.152843   
                2      1.148195  1.148195  1.148195  1.148195  1.148195   
                3      1.219923  1.219923  1.219923  1.219923  1.219923   
                4      1.185677  1.185677  1.185677  1.185677  1.185677   
                5      1.169839  1.169839  1.169839  1.169839  1.169839   
...                         ...       ...       ...       ...       ...   
9       3       20     1.152782  1.152782  1.152782  1.152782  1.152782   
                21     1.200473  1.200473  1.200473  1.200473  1.200473   
                22     1.174676  1.174676  1.174676  1.174676  1.174676   
                23     1.175657  1.175657  1.175657  1.175657  1.175657   
                24     1.176242  1.176242  1.176242  1.176242  1.176242   

                           1499      1500      1501      1502  emotion  
id_user session video                                                   
10      1       1      1.152843  1.152843  1.152843  1.152843        1  
                2      1.148195  1.148195  1.148195  1.148195        2  
                3      1.219923  1.219923  1.219923  1.219923        3  
                4      1.185677  1.185677  1.185677  1.185677        0  
                5      1.169839  1.169839  1.169839  1.169839        2  
...                         ...       ...       ...       ...      ...  
9       3       20     1.152782  1.152782  1.152782  1.152782        0  
                21     1.200473  1.200473  1.200473  1.200473        2  
                22     1.174676  1.174676  1.174676  1.174676        0  
                23     1.175657  1.175657  1.175657  1.175657        1  
                24     1.176242  1.176242  1.176242  1.176242        0  

[1080 rows x 1504 columns]

### Subject Dependent

In [95]:
scores_test = []
scores_training = []
clfSD = GradientBoostingClassifier(n_estimators=80, learning_rate=0.001, max_depth=5,random_state=104)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    clfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(clfSD.score(X_testSD, y_testSD))
    scores_training.append(clfSD.score(X_trainSD, y_trainSD))

In [96]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.17777777777777778
TRAIN: 0.6569444444444444


In [97]:
scores_test = []
scores_training = []
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42)
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    rfSD.fit(X_trainSD, y_trainSD)
    scores_test.append(rfSD.score(X_testSD, y_testSD))
    scores_training.append(rfSD.score(X_trainSD, y_trainSD))

In [98]:
print("TEST:", np.array(scores_test).mean())
print("TRAIN:", np.array(scores_training).mean())

TEST: 0.25277777777777777
TRAIN: 0.9847222222222222


### Subject indipendent

In [ ]:
scores_test = []
gbc_SI = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)
for i in range(1,4):
    X_SI, y_SI, groups = subject_independent_split(df,i)
    scores_gbc = cross_val_score(gbc_SI, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(scores_gbc.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  5.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
print("TEST:", np.array(scores_test).mean())

In [ ]:
scores_test = []
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    scores_rfc = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores_rfc.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())

### Subject Biased

In [ ]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", clf.score(X_testSB, y_testSB))
print("TEST", clf.score(X_trainSB, y_trainSB))

In [ ]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [ ]:
print("TEST", rfc.score(X_testSB, y_testSB))
print("TEST", rfc.score(X_trainSB, y_trainSB))